Um dos aplicativos mais poderosos habilitados por LLMs são os sofisticados chatbots de perguntas e respostas (Q&A). Esses são aplicativos que podem responder perguntas sobre informações de fontes específicas. 

# O que é RAG?
RAG é uma técnica para aumentar o conhecimento do LLM com dados adicionais.

Se você quiser construir aplicativos de IA que possam raciocinar sobre dados privados ou dados introduzidos após a data de corte de um modelo, você precisa aumentar o conhecimento do modelo com as informações específicas de que ele precisa. O processo de trazer as informações apropriadas e inseri-las no prompt do modelo é conhecido como Retrieval Augmented Generation (RAG).

Uma aplicação RAG típica tem dois componentes principais:

* <strong>Indexação</strong> : um pipeline para ingerir dados de uma fonte e indexá-los. Isso geralmente acontece offline.

* <strong>Recuperação e geração</strong> : a cadeia RAG real, que recebe a consulta do usuário em tempo de execução e recupera os dados relevantes do índice, passando-os então para o modelo.

## Indexação
A sequência completa mais comum de dados brutos até a resposta se parece com

1. Carregar : Primeiro precisamos carregar nossos dados. Isso é feito com Document Loaders .
2. Split : divisores de texto quebram grandes Documentsem pedaços menores. Isso é útil tanto para indexar dados quanto para passá-los para um modelo, já que pedaços grandes são mais difíceis de pesquisar e não cabem na janela de contexto finita de um modelo.
3. Store : Precisamos de um lugar para armazenar e indexar nossas divisões, para que elas possam ser pesquisadas mais tarde. Isso geralmente é feito usando um modelo VectorStore e Embeddings .

## Recuperação e geração
1. Recuperar : Dada uma entrada do usuário, as divisões relevantes são recuperadas do armazenamento usando um Retriever .
2. Gerar : Um ChatModel / LLM produz uma resposta usando um prompt que inclui a pergunta e os dados recuperados

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

 ········


In [4]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\Kaique\Desktop\langchain-study\env\Lib\site-packages\langsmith\client.py:5402: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


'Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. It often involves prompting the model to think step by step, allowing it to tackle each part systematically. Techniques like Chain of Thought (CoT) and Tree of Thoughts enhance this process by exploring various reasoning possibilities at each stage.'

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
